In [1]:
import arcpy
from arcpy import env
import os
from arcgis import GIS
from arcgis.features import GeoAccessor
import pandas as pd
import numpy as np

arcpy.env.overwriteOutput = True
arcpy.env.parallelProcessingFactor = "90%"

# show all columns
pd.options.display.max_columns = None

# pd.DataFrame.spatial.from_featureclass(???)
# df.spatial.to_featureclass(location=???,sanitize_columns=False)

## Inputs

In [2]:
parcels = r".\Inputs\WFRC_Non_Residential_Parcels.gdb\Non_Residential_Parcels"
hui = r"E:\Projects\Housing-Unit-Inventory\Current_Version\wfrc_mag_housing_inventory_2020_20220209.gdb\housing_inventory_2020_20220209"
mag_parcels = r'.\Inputs\MAG_REMM_Parcels_for_Splitting.gdb\MAG_REMM_Parcels_for_splitting'
raw_parcel_pts = r'.\Inputs\parcel_centroids.gdb\wfrc_centroids'
addr_pts = r'.\Inputs\addr_pts.gdb\addr_pts_wfrc_mag'
old_taz = r'.\Inputs\TAZ_832_ID_Only.shp'
new_taz = r'.\Inputs\TAZ_900_ID_Only.shp'
census_tracts = r'.\Inputs\tl_2018_49_tract.shp'
centers = r'.\Inputs\WFRC_Centers_MaxFAR.shp'
nonres_pts = r'.\Inputs\remm_base_year_2015.gdb\nonres_sqft_2015_pts'
base_year_surface1 = r".\Inputs\zoning_surface.gdb\base_year_surface_far"
base_year_surface2 = r".\Inputs\zoning_surface.gdb\base_year_surface_maxdua_year_built"

## Output directories

In [3]:
if not os.path.exists('Outputs'):
    os.makedirs('Outputs')
    
outputs = ['.\\Outputs', "temp_file.gdb", 'remm_base_year_2019.gdb']
gdb = os.path.join(outputs[0], outputs[1])
gdb2 = os.path.join(outputs[0], outputs[2])

if not arcpy.Exists(gdb):
    arcpy.CreateFileGDB_management(outputs[0], outputs[1])

if not arcpy.Exists(gdb2):
    arcpy.CreateFileGDB_management(outputs[0], outputs[2])

In [4]:
# ~5 min
hui_sdf = pd.DataFrame.spatial.from_featureclass(hui)
hui_sdf = hui_sdf[['TYPE', 'IS_OUG', 'COUNTY', 'UNIT_COUNT', 'DUA','ACRES', 'TOT_BD_FT2', 'TOT_VALUE', 'APX_BLT_YR', 'UNIT_ID', 'SHAPE']].copy() # add floor count back later, can use centroids from parcels
hui_sdf.columns = ['TYPE', 'IS_OUG', 'COUNTY_NAME', 'UNIT_COUNT', 'DUA','PARCEL_ACRES', 'BLDG_SQFT', 'TOTAL_MKT_VALUE', 'BUILT_YR','UNIT_ID', 'SHAPE']
hui_sdf['UNIT_ID'] = hui_sdf.index + 1
hui_copy =  os.path.join(gdb, '_00_hui_copy')
hui_sdf.spatial.to_featureclass(location=hui_copy,sanitize_columns=False)

'e:\\Projects\\REMM-Input-Data-Prep-2019\\Parcels\\2020-WFRC\\Outputs\\temp_file.gdb\\_00_hui_copy'

In [5]:
# copy hui
# hui_copy = arcpy.FeatureClassToFeatureClass_conversion(hui, gdb, '_00_hui_copy')
arcpy.CalculateField_management(hui_copy, "TOTAL_MKT_VALUE", """0""", "PYTHON3")

# use spatial join to summarize TOTAL_MKT_VALUE, LAND_MKT_VALUE, BLDG_SQFT
target_features = hui_copy
join_features = raw_parcel_pts
output_features = os.path.join(gdb, "_00_hui_pc_sj")

fieldmappings = arcpy.FieldMappings()
fieldmappings.addTable(target_features)
fieldmappings.addTable(join_features)

# attribute to summarize
fieldindex = fieldmappings.findFieldMapIndex('TOTAL_MKT_VALUE')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'Sum'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

fieldindex = fieldmappings.findFieldMapIndex('LAND_MKT_VALUE')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'Sum'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

fieldindex = fieldmappings.findFieldMapIndex('BLDG_SQFT')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'Sum'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# run the spatial join
sj = arcpy.SpatialJoin_analysis(target_features, join_features, output_features,'JOIN_ONE_TO_ONE', "KEEP_ALL", 
                           fieldmappings, "INTERSECT")

In [6]:
# post process hui
sj_sdf = pd.DataFrame.spatial.from_featureclass(sj[0])
sj_sdf = sj_sdf[['UNIT_ID', 'TOTAL_MKT_VALUE','LAND_MKT_VALUE','BLDG_SQFT', 'COUNTY_NAME']].copy()
sj_sdf.columns = ['UNIT_ID', 'TOTAL_MKT_VALUE_NEW','LAND_MKT_VALUE','BLDG_SQFT_NEW', 'COUNTY_NAME']

In [7]:
merged = hui_sdf.merge(sj_sdf, left_on=['UNIT_ID','COUNTY_NAME'],right_on=['UNIT_ID','COUNTY_NAME'],how='left')
merged['PARCEL_ACRES'] = pd.to_numeric(merged['PARCEL_ACRES'].round(4)) 

# recalc the building type
merged.loc[(merged['TYPE'] == 'single_family'), 'building_type_id'] = 1
merged.loc[(merged['TYPE'] == 'multi_family'), 'building_type_id'] = 2

# if current value is zero, update
merged.loc[((merged['TOTAL_MKT_VALUE'] == 0) | (merged['TOTAL_MKT_VALUE'].isnull() == True)) & 
            ((merged['TOTAL_MKT_VALUE_NEW'] > 0) & (merged['TOTAL_MKT_VALUE_NEW'] < merged['TOTAL_MKT_VALUE'])), 'TOTAL_MKT_VALUE'] = merged['TOTAL_MKT_VALUE_NEW']

# merged['TOTAL_MKT_VALUE'] =  merged['TOTAL_MKT_VALUE_NEW']

merged.loc[((merged['BLDG_SQFT'] == 0) | (merged['BLDG_SQFT'].isnull() == True)) & 
            (merged['BLDG_SQFT_NEW'] != 0), 'BLDG_SQFT'] = merged['BLDG_SQFT_NEW']


In [8]:
merged.shape

(391838, 15)

In [9]:
hui_post_process = os.path.join(gdb, "_01_hui_post_process")
merged.spatial.to_featureclass(location=hui_post_process,sanitize_columns=False)

'e:\\Projects\\REMM-Input-Data-Prep-2019\\Parcels\\2020-WFRC\\Outputs\\temp_file.gdb\\_01_hui_post_process'

In [10]:
# select all parcels within buffered housing unit inventory parcels and delete
hui_buff = arcpy.Buffer_analysis(hui_post_process, os.path.join(gdb, "_02_hui_buffer_1_25m"), '1.25 METERS')

In [11]:
# select and remove parcels already existing in the housing inventory
parcels_copy = arcpy.FeatureClassToFeatureClass_conversion(parcels, gdb, '_03_parcels_copy')
parcels_with_selection = arcpy.SelectLayerByLocation_management(parcels_copy, 'WITHIN', hui_buff)
arcpy.FeatureClassToFeatureClass_conversion(parcels_with_selection, gdb, '_02_parcels_removed')
arcpy.DeleteFeatures_management(parcels_with_selection)

<Result '_03_parcels_copy_Layer2'>

In [12]:
# add reforamtted hui parcels to main parcel dataset
parcels_reunited = arcpy.Merge_management([parcels_with_selection, hui_post_process], os.path.join(gdb, '_04_WFRC_Reunited_Parcels'))

# Eliminate WFRC Road Polygons


In [13]:
parcels_reunited_lyr = arcpy.MakeFeatureLayer_management(parcels_reunited, 'parcels_split_merge_lyr') 
arcpy.AddField_management(parcels_reunited_lyr, 'Thinness', 'FLOAT')
arcpy.CalculateField_management(parcels_reunited_lyr, field='Thinness', expression="(4 * math.pi* !Shape_Area!) / (!Shape_Length! * !Shape_Length!)", expression_type="PYTHON3")
query = """(Thinness < 0.26 And building_type_id NOT IN (1, 2,3, 5, 4, 10, 3, 7, 8,9,10,11,12,13, 14, 15)) And PARCEL_ACRES > 0.025 And 
            PARCEL_ID NOT IN ('22081850140000', '15011090062000','110810001') And agriculture <> 1"""

arcpy.SelectLayerByAttribute_management(parcels_reunited_lyr, 'NEW_SELECTION', query)

# exclude parcels that overlap with pts from selection
do_not_delete = '.\inputs\do_not_delete_thinness.shp'
arcpy.SelectLayerByLocation_management(in_layer=parcels_reunited_lyr, overlap_type="INTERSECT",
                                       select_features=do_not_delete,
                                       selection_type='REMOVE_FROM_SELECTION')

parcels_reunited_lyr = arcpy.DeleteFeatures_management(parcels_reunited_lyr)

# Merge with MAG Parcels

In [14]:
# standardize
mag_parcels_sdf = pd.DataFrame.spatial.from_featureclass(mag_parcels)
mag_parcels_sdf.loc[(mag_parcels_sdf['OUG_ID']>0), 'IS_OUG'] = '1'
mag_parcels_sdf.loc[(mag_parcels_sdf['OUG_ID']==0), 'IS_OUG'] = '0'
mag_parcels_sdf = mag_parcels_sdf.rename(columns={"LAND_VAL": "LAND_MKT_VALUE"})
mag_parcels_sdf = mag_parcels_sdf.rename(columns={"TOT_VAL": "TOTAL_MKT_VALUE"})
mag_parcels_sdf = mag_parcels_sdf.rename(columns={"residential_units": "UNIT_COUNT"})
mag_parcels_sdf = mag_parcels_sdf.rename(columns={"building_sqft": "BLDG_SQFT"})
mag_parcels_sdf = mag_parcels_sdf.rename(columns={"year_built": "BUILT_YR"})
mag_parcels_sdf = mag_parcels_sdf.rename(columns={"ACREAGE": "PARCEL_ACRES"})
mag_parcels_sdf = mag_parcels_sdf.rename(columns={"Parent_Parcel": "PARCEL_ID"})
mag_parcels_sdf['PARCEL_ID'] = mag_parcels_sdf['PARCEL_ID'].astype(str)

mag_parcels_sdf['COUNTY_NAME'] = 'Utah'
mag_parcels_sdf['COUNTY_ID'] = 49

# set parcels built after 2019 to empty
mag_parcels_sdf.loc[(mag_parcels_sdf['BUILT_YR']>2019), 'TOTAL_MKT_VALUE'] = mag_parcels_sdf['LAND_MKT_VALUE']
mag_parcels_sdf.loc[(mag_parcels_sdf['BUILT_YR']>2019), 'building_type_id'] = 0
mag_parcels_sdf.loc[(mag_parcels_sdf['BUILT_YR']>2019), 'UNIT_COUNT'] = 0
mag_parcels_sdf.loc[(mag_parcels_sdf['BUILT_YR']>2019), 'BLDG_SQFT'] = 0

mag_parcels_sdf = mag_parcels_sdf[['COUNTY_NAME','COUNTY_ID',"TOTAL_MKT_VALUE",'IS_OUG','building_type_id',"LAND_MKT_VALUE", 
                                    "UNIT_COUNT","BLDG_SQFT","BUILT_YR","PARCEL_ACRES","PARCEL_ID", 'parcel_id_REMM', "SHAPE"]].copy()

mag_parcels_sdf.spatial.to_featureclass(location=os.path.join(gdb, "_04_mag_parcels_processed"),sanitize_columns=False)

'e:\\Projects\\REMM-Input-Data-Prep-2019\\Parcels\\2020-WFRC\\Outputs\\temp_file.gdb\\_04_mag_parcels_processed'

In [15]:
# merge wfrc and mag parcels
wfrc_mag_parcels = arcpy.Merge_management([parcels_reunited_lyr, os.path.join(gdb, "_04_mag_parcels_processed")], os.path.join(gdb, '_05_wfrc_mag_parcels'))
arcpy.CalculateField_management(wfrc_mag_parcels, 'parcel_id_REMM', """!OBJECTID!""", "PYTHON3")


<Result '.\\Outputs\\temp_file.gdb\\_05_wfrc_mag_parcels'>

## Set Agriculture attribute (check this!)

In [16]:
# set land use type on parcels using glfu
ag = r'.\Inputs\agriculture_2020.shp'

wfrc_mag_parcels_lyr = arcpy.MakeFeatureLayer_management(wfrc_mag_parcels, 'wfrc_mag_parcels_lyr') 
arcpy.management.SelectLayerByLocation(wfrc_mag_parcels_lyr, "HAVE_THEIR_CENTER_IN", select_features=ag, selection_type="NEW_SELECTION")

arcpy.CalculateField_management(wfrc_mag_parcels_lyr, field='agriculture', expression="1", expression_type="PYTHON3")

arcpy.SelectLayerByAttribute_management(wfrc_mag_parcels_lyr, 'SWITCH_SELECTION')

arcpy.CalculateField_management(wfrc_mag_parcels_lyr, field='agriculture', expression="0", expression_type="PYTHON3")

<Result 'wfrc_mag_parcels_lyr'>

In [17]:
# clear some memory
del hui_sdf
del merged
# del wfrc_parcels_sdf
del mag_parcels_sdf
# del wfrc_mag_parcels_sdf
del parcels_reunited_lyr
del parcels_reunited

# Split Parcels

In [18]:
env.outputCoordinateSystem = arcpy.SpatialReference('NAD 1983 UTM Zone 12N')
desc = arcpy.Describe(wfrc_mag_parcels)

#create the acre mesh- in this case, the cell_width and height calculate to 1 acre.
acre_mesh = arcpy.management.CreateFishnet(out_feature_class = os.path.join(gdb,'_05a_acre_mesh'),
    origin_coord = str(desc.extent.lowerLeft),
    y_axis_coord = str(desc.extent.XMin) + " " + str(desc.extent.YMax + 10),
    cell_width = 63.6, # meters
    cell_height = 63.6, # meters
    number_rows = '#',
    number_columns = '#',
    labels = 'NO_LABELS', 
    template = wfrc_mag_parcels, 
    geometry_type = 'POLYGON'
)

#Add Required Fields: RealBuildingID
arcpy.management.AddFields(in_table=acre_mesh, field_description=[["RealBldg", "DOUBLE", "RealBldg", "", "", ""]])

<Result '.\\Outputs\\temp_file.gdb\\_05a_acre_mesh'>

In [19]:
#Identifying which parcels meet the qualifications
Parcels_7Acre_wBuildings = arcpy.conversion.FeatureClassToFeatureClass(in_features=wfrc_mag_parcels, 
                                                                       out_path=gdb, out_name="_06_Parcels_7Acre_wBuildings", 
                                                                       where_clause="(building_type_id IN (0, 1, 15)) And (PARCEL_ACRES >= 7) AND (IS_OUG <>  '1')")

#Identifying which buildings reside on those parcels that meet qualifications
Buildings_7Acre_wBuildings = arcpy.analysis.SpatialJoin(target_features=addr_pts, 
                                                        join_features=Parcels_7Acre_wBuildings, 
                                                        out_feature_class=os.path.join(gdb,'_07_Buildings_7Acre_wBuildings'), 
                                                        join_operation="JOIN_ONE_TO_ONE", 
                                                        join_type="KEEP_COMMON",
                                                        match_option="INTERSECT", 
                                                        search_radius="", 
                                                        distance_field_name="")

#Adding the Preserve Bldg field- used to identify which building areas to preserve
arcpy.management.AddField(in_table=Buildings_7Acre_wBuildings, 
                                                       field_name="PreserveBldg", 
                                                       field_type="SHORT", 
                                                       field_precision=None, 
                                                       field_scale=None, 
                                                       field_length=None, 
                                                       field_alias="RealBldg", 
                                                       field_is_nullable="NULLABLE", 
                                                       field_is_required="NON_REQUIRED", 
                                                       field_domain="")

arcpy.management.CalculateField(in_table=Buildings_7Acre_wBuildings, 
                                                    field="PreserveBldg", 
                                                    expression="1", 
                                                    expression_type="ARCADE", 
                                                    code_block="", 
                                                    field_type="TEXT")

Preserve_Buildings = arcpy.conversion.FeatureClassToFeatureClass(in_features=Buildings_7Acre_wBuildings, 
                                                                       out_path=gdb, out_name="_08_Preserve_Buildings")

In [20]:
# split mesh grids by parcel geometry to prevent overlap into neighboring parcels
Mesh_Selected_Parcels_Identity = arcpy.Identity_analysis (acre_mesh, Parcels_7Acre_wBuildings, os.path.join(gdb,'_09_Mesh_Selected_Parcels_Identity'))

#Use Spatial Join to identify which acre_mesh pieces contain those preserve_buildings
Mesh_w_Buildings = arcpy.analysis.SpatialJoin(target_features=Mesh_Selected_Parcels_Identity, 
                           join_features=Preserve_Buildings, 
                           out_feature_class=os.path.join(gdb,'_09_Mesh_w_Buildings'), 
                           join_operation="JOIN_ONE_TO_ONE", 
                           join_type="KEEP_ALL",
                           match_option="INTERSECT", 
                           search_radius="", 
                           distance_field_name="")



#convert to new Feature Class if Preserve_blg = 1
Mesh_w_PreserveBldgs = arcpy.conversion.FeatureClassToFeatureClass(in_features=Mesh_w_Buildings, 
                                                                       out_path=gdb, out_name="_10_Mesh_w_PreserveBldgs", 
                                                                       where_clause="PreserveBldg IN (1)")

In [21]:
#Select the parcels with buildings on them and prep for mesh erasure and convert to feature class
Parcels_for_Erase_select = arcpy.management.SelectLayerByLocation(in_layer=[wfrc_mag_parcels],
                                                                 overlap_type="INTERSECT", 
                                                                 select_features=Buildings_7Acre_wBuildings, 
                                                                 search_distance="", 
                                                                 selection_type="NEW_SELECTION", 
                                                                 invert_spatial_relationship="NOT_INVERT")

# ensure undesired parcels aren't selected
arcpy.SelectLayerByAttribute_management(Parcels_for_Erase_select, 'SUBSET_SELECTION', "(building_type_id IN (0, 1, 15)) And (PARCEL_ACRES >= 7) AND (IS_OUG <>  '1')")


Parcels_for_Erase= arcpy.conversion.FeatureClassToFeatureClass(in_features=Parcels_for_Erase_select, 
                                                                       out_path=gdb, out_name="_11_Parcels_for_Erase")

In [22]:
parcels_erased = arcpy.analysis.Erase(in_features=Parcels_for_Erase,
                                      erase_features=Mesh_w_PreserveBldgs, 
                                      out_feature_class= os.path.join(gdb,'_12_parcels_erased'),
                                      cluster_tolerance="")

In [23]:
# calculate all building parcel attributes to zero out for the non-building pieces
#YEAR BUILT
parcels_erased = arcpy.management.CalculateField(in_table=parcels_erased, 
                                                                  field="BUILT_YR", 
                                                                  expression="""0""", 
                                                                  expression_type="PYTHON3")

#BUILDING SQFT
parcels_erased = arcpy.management.CalculateField(in_table=parcels_erased, 
                                                                  field="BLDG_SQFT", 
                                                                  expression="""0""", 
                                                                  expression_type="PYTHON3")

#BUILDING TYPE ID
parcels_erased = arcpy.management.CalculateField(in_table=parcels_erased, 
                                                                  field="building_type_id", 
                                                                  expression="""0""", 
                                                                  expression_type="PYTHON3")

In [24]:
#join erased parcels to original parcels- puzzle piece 1
parcels_with_structures = arcpy.analysis.SymDiff(Parcels_for_Erase, parcels_erased, os.path.join(gdb,'_13_parcels_with_structures'))
arcpy.AddField_management(parcels_with_structures, 'Split', 'LONG')
arcpy.CalculateField_management(parcels_with_structures, field='Split', expression=2, expression_type="PYTHON3") 

parcels_erased_union = arcpy.management.Merge(inputs=[parcels_erased, parcels_with_structures],
                                           output=os.path.join(gdb,'_14_parcels_erased_union'),
                                           add_source="NO_SOURCE_INFO")

In [25]:
# #All other parcels not included in the parcels_erased_union
Other_parcels_select = arcpy.management.SelectLayerByLocation(in_layer=[wfrc_mag_parcels], 
                                                          overlap_type="INTERSECT", 
                                                          select_features=Buildings_7Acre_wBuildings, 
                                                          search_distance="", 
                                                          selection_type="NEW_SELECTION", 
                                                          invert_spatial_relationship="INVERT")

# Other_parcels_select = arcpy.SelectLayerByAttribute_management(wfrc_mag_parcels_w_TAZID, 'NEW_SELECTION', "(building_type_id IN (0, 1, 15)) And (PARCEL_ACRES >= 7) AND (IS_OUG <>  '1')",  invert_where_clause="INVERT")

#convert selection to new feature class
All_other_parcels= arcpy.conversion.FeatureClassToFeatureClass(in_features=Other_parcels_select, 
                                                                       out_path=gdb, 
                                                               out_name="_15_All_other_parcels")

In [26]:
#merging the non-erased, and the erased parcels back together
prepared_parcels = arcpy.management.Merge(inputs=[All_other_parcels, parcels_erased_union],
                                           output=os.path.join(gdb,'_16_prepared_parcels'),
                                           add_source="NO_SOURCE_INFO")

In [27]:
#select parcels with acreage larger than 7 or haven't been erased
parcels_YES_split_select = arcpy.management.SelectLayerByAttribute(in_layer_or_view=prepared_parcels,
                                                                               selection_type="NEW_SELECTION", 
                                                                               where_clause=""" building_type_id IN (0, 1, 15) And (PARCEL_ACRES >= 7) AND (IS_OUG <>  '1') And 
                                                                                               (FID__12_parcels_erased > -1 Or FID__12_parcels_erased IS NULL)""", 
                                                                               invert_where_clause="")

parcels_YES_split = arcpy.conversion.FeatureClassToFeatureClass(in_features=parcels_YES_split_select, 
                                                                       out_path=gdb, 
                                                               out_name="_17_parcels_YES_split")

arcpy.CalculateField_management(parcels_YES_split, field='Split', expression=1, expression_type="PYTHON3") 

# WRONG PLACE apply taz to splitting process
# taz_layer = arcpy.MakeFeatureLayer_management(new_taz, 'new_taz') 
# query = '''REMM IN (1)'''
# arcpy.management.SelectLayerByAttribute(taz_layer, "NEW_SELECTION", query)
parcels_YES_split_taz = arcpy.Identity_analysis(parcels_YES_split, new_taz, os.path.join(gdb,'_18_parcels_YES_split_taz'))

In [28]:
parcels_split = arcpy.management.SubdividePolygon(in_polygons=parcels_YES_split_taz, 
                                  out_feature_class= os.path.join(gdb,'_19_parcels_split'), 
                                  method="EQUAL_AREAS", 
                                  num_areas=None, 
                                  target_area="5 Acres", 
                                  target_width="", 
                                  split_angle=0, 
                                  subdivision_type="STACKED_BLOCKS")

In [29]:
#select all other parcels that arent ready for splitting
parcels_NO_split_select = arcpy.management.SelectLayerByAttribute(in_layer_or_view=prepared_parcels,
                                                                               selection_type="NEW_SELECTION", 
                                                                               where_clause="""building_type_id IN (0, 1, 15) And (PARCEL_ACRES >= 7) AND (IS_OUG <>  '1')
                                                                                              And (FID__12_parcels_erased > -1 Or FID__12_parcels_erased IS NULL)""", 
                                                                               invert_where_clause="INVERT")


parcels_NO_split = arcpy.conversion.FeatureClassToFeatureClass(in_features=parcels_NO_split_select, 
                                                                       out_path=gdb, 
                                                               out_name="_20_parcels_NO_split")

In [30]:
#merging all parcels back together again
parcels_split_merge = arcpy.management.Merge(inputs=[parcels_split, parcels_NO_split],
                                           output=os.path.join(gdb,'_21_parcels_split_merge'),
                                           add_source="NO_SOURCE_INFO")

# recalc acreage
arcpy.AddField_management(parcels_split_merge, 'PARCEL_ACRES_NEW', 'FLOAT')
arcpy.CalculateField_management(parcels_split_merge, "PARCEL_ACRES_NEW", """!SHAPE.area@ACRES!""", "PYTHON3")
# arcpy.CalculateGeometryAttributes_management(parcels_split_merge, ["PARCEL_ACRES_NEW", "AREA"], area_unit="ACRES") # bugged

# create layer
parcels_split_merge_lyr = arcpy.MakeFeatureLayer_management(parcels_split_merge, 'parcels_split_merge_lyr') 

In [31]:
# delete tiny parcels
query = """ Shape_Area < 15 """
arcpy.SelectLayerByAttribute_management(parcels_split_merge_lyr, 'NEW_SELECTION', query)
parcels_split_merge_lyr = arcpy.DeleteFeatures_management(parcels_split_merge_lyr)
arcpy.SelectLayerByAttribute_management(parcels_split_merge_lyr, 'CLEAR_SELECTION', query)

<Result 'parcels_split_merge_lyr'>

In [32]:
# set land use type on parcels using glfu
gflu = '.\Inputs\WFRC_MAG_Base_Policy.shp'
gflu_lyr = arcpy.MakeFeatureLayer_management(gflu, 'gflu_lyr') 


##################
# Type 1 (SF)
##################

# query for land use type
query = (""" GenLUType IN ('Any Development','Any Residential', 'Mixed Use SF', 'Residential SF', 'Residential SF/Retail') 
                Or (GenLUType = 'Residential' And MaxDUA <= 12)
                Or (GenLUType = 'Agricultural' And MaxDUA < .2)
                Or (GenLUType = 'Mixed Use' And MaxDUA <= 18)  """)
arcpy.SelectLayerByAttribute_management(gflu_lyr, 'NEW_SELECTION', query)


arcpy.management.SelectLayerByLocation(parcels_split_merge_lyr, "HAVE_THEIR_CENTER_IN", gflu_lyr, None, "NEW_SELECTION", "NOT_INVERT")

arcpy.CalculateField_management(parcels_split_merge_lyr, field='type1', expression="'t'",
                                expression_type="PYTHON3")


##################
# Type 2 (MF)
##################

# query for land use type
query = (""" (GenLUType IN ('Any Residential', 'Any Development', 'Industrial/Mixed Use MF', 'Mixed Use', 
                          'Residential MF/Office', 'Residential MF', 'Mixed Use MF', 'Mixed Use SF', 'Residential/Office', 
                          'Residential/Retail')) 
                          Or (GenLUType = 'Residential' And MaxDUA >= 8)
                           """)
arcpy.SelectLayerByAttribute_management(gflu_lyr, 'NEW_SELECTION', query)

# select the parcels
arcpy.management.SelectLayerByLocation(parcels_split_merge_lyr, "HAVE_THEIR_CENTER_IN", gflu_lyr, None, "NEW_SELECTION", "NOT_INVERT")

arcpy.CalculateField_management(parcels_split_merge_lyr, field='type2', expression="'t'",
                                expression_type="PYTHON3")

#######################
# Type 3 (Industrial)
#######################

# query for land use type
query = """ GenLUType IN ('Industrial', 'Industrial/Office', 'Any Development', 'Industrial/Mixed Use MF','Business Park') """
arcpy.SelectLayerByAttribute_management(gflu_lyr, 'NEW_SELECTION', query)

# select the parcels
arcpy.management.SelectLayerByLocation(parcels_split_merge_lyr, "HAVE_THEIR_CENTER_IN", gflu_lyr, None, "NEW_SELECTION", "NOT_INVERT")

arcpy.CalculateField_management(parcels_split_merge_lyr, field='type3', expression="'t'",
                                expression_type="PYTHON3")

#######################
# Type 4 (Retail)
#######################

# query for land use type
query = """ GenLUType IN ('Any Development', 'Industrial/Retail', 'Retail', 'Retail/Office', 'Residential/Retail', 
                          'Residential SF/Retail', 'Mixed Use', 'Commercial') """
arcpy.SelectLayerByAttribute_management(gflu_lyr, 'NEW_SELECTION', query)

# select the parcels
arcpy.management.SelectLayerByLocation(parcels_split_merge_lyr, "HAVE_THEIR_CENTER_IN", gflu_lyr, None, "NEW_SELECTION", "NOT_INVERT")

arcpy.CalculateField_management(parcels_split_merge_lyr, field='type4', expression="'t'",
                                expression_type="PYTHON3")


#######################
# Type 5 (Office)
#######################

# query for land use type
query = """ GenLUType IN ('Retail/Office', 'Office', 'Any Commercial', 'Residential/Office', 'Residential MF/Office','Commercial','Business Park', 'Mixed Use') """
arcpy.SelectLayerByAttribute_management(gflu_lyr, 'NEW_SELECTION', query)

# select the parcels
arcpy.management.SelectLayerByLocation(parcels_split_merge_lyr, "HAVE_THEIR_CENTER_IN", gflu_lyr, None, "NEW_SELECTION", "NOT_INVERT")

arcpy.CalculateField_management(parcels_split_merge_lyr, field='type5', expression="'t'",
                                expression_type="PYTHON3")

####################################
# Type 6 (Government and Education)
###################################

# query for land use type
query = """ GenLUType IN ('Any Development', 'Government/Education', 'Business Park') """
arcpy.SelectLayerByAttribute_management(gflu_lyr, 'NEW_SELECTION', query)

# select the parcels
arcpy.management.SelectLayerByLocation(parcels_split_merge_lyr, "HAVE_THEIR_CENTER_IN", gflu_lyr, None, "NEW_SELECTION", "NOT_INVERT")

arcpy.CalculateField_management(parcels_split_merge_lyr, field='type6', expression="'t'",
                                expression_type="PYTHON3")


####################################
# Type 7 (Mixed Use)
###################################

# query for land use type
query = """ GenLUType IN ('Any Development', 'Industrial/Mixed Use MF', 'Mixed Use', 'Mixed Use MF', 'Mixed Use SF', 
                          'Residential MF/Office', 'Residential SF/Retail', 'Residential/Office', 'Residential/Retail', 
                          'Retail/Office') """
arcpy.SelectLayerByAttribute_management(gflu_lyr, 'NEW_SELECTION', query)

# select the parcels
arcpy.management.SelectLayerByLocation(parcels_split_merge_lyr, "HAVE_THEIR_CENTER_IN", gflu_lyr, None, "NEW_SELECTION", "NOT_INVERT")

arcpy.CalculateField_management(parcels_split_merge_lyr, field='type7', expression="'t'",
                                expression_type="PYTHON3")

####################################
# Type 8 (Other)
###################################

# # query for land use type
# query = """ GenLUType IN ('Any Development', 'Industrial/Retail', 'Retail', 'Retail/Office', 'Residential/Retail', 
#                           'Residential SF/Retail') """
# arcpy.SelectLayerByAttribute_management(gflu_lyr, 'NEW_SELECTION', query)

# # select the parcels
# arcpy.SelectLayerByLocation_management(in_layer=merged_parcels_lyr,overlap_type="INTERSECT",
#                                        select_features=gflu_lyr,
#                                        selection_type='NEW_SELECTION')

# arcpy.CalculateField_management(merged_parcels_lyr, field='type8', expression="'t'".format(tag),
#                                 expression_type="PYTHON3")

#################################
# Undevelopable
#################################

# query for land use type
query = """ GenLUType IN ('NoBuild', 'Open Space', 'Sensitive Areas','Transportation') """
arcpy.SelectLayerByAttribute_management(gflu_lyr, 'NEW_SELECTION', query)

# select the parcels
arcpy.management.SelectLayerByLocation(parcels_split_merge_lyr, "HAVE_THEIR_CENTER_IN", gflu_lyr, None, "NEW_SELECTION", "NOT_INVERT")

arcpy.CalculateField_management(parcels_split_merge_lyr, field='NoBuild', expression="1",
                                expression_type="PYTHON3")


# clear selected parcels
arcpy.SelectLayerByAttribute_management(parcels_split_merge_lyr, 'CLEAR_SELECTION', query)
arcpy.SelectLayerByAttribute_management(gflu_lyr, 'CLEAR_SELECTION', query)

<Result 'gflu_lyr'>

In [50]:
#attribute cleanup
final_parcel_df = pd.DataFrame.spatial.from_featureclass(parcels_split_merge[0])

# Adjust non-true zoning to false
final_parcel_df.loc[(final_parcel_df['type1'] != 't'), 'type1'] = 'f'
final_parcel_df.loc[(final_parcel_df['type2'] != 't'), 'type2'] = 'f'
final_parcel_df.loc[(final_parcel_df['type3'] != 't'), 'type3'] = 'f'
final_parcel_df.loc[(final_parcel_df['type4'] != 't'), 'type4'] = 'f'
final_parcel_df.loc[(final_parcel_df['type5'] != 't'), 'type5'] = 'f'
final_parcel_df.loc[(final_parcel_df['type6'] != 't'), 'type6'] = 'f'
final_parcel_df.loc[(final_parcel_df['type7'] != 't'), 'type7'] = 'f'
final_parcel_df.loc[(final_parcel_df['type8'] != 't'), 'type8'] = 'f'

# divide land value by this number for new approximate value
final_parcel_df['Split_Factor'] = final_parcel_df['PARCEL_ACRES']/final_parcel_df['PARCEL_ACRES_NEW']
final_parcel_df['PARCEL_ACRES'] = final_parcel_df['PARCEL_ACRES_NEW']

fields = ['parcel_id_REMM','COUNTY_NAME','TAZID', 'COUNTY_ID', 'PARCEL_ID', 'TOTAL_MKT_VALUE', 'LAND_MKT_VALUE', 'UNIT_COUNT', 'BLDG_SQFT', 'FLOORS_CNT',
         'BUILT_YR', 'EFFBUILT_YR', 'IS_OUG','max_dua', 'max_far', 'max_height', 'type1', 'type2', 'type3', 'type4', 'type5', 'type6', 
         'type7','type8', 'agriculture', 'basebldg', 'NoBuild', 'redev_friction', 'building_type_id', 'x', 'y', 
        'PARCEL_ACRES', 'Split','Split_Factor', 'SHAPE']

# save old parcel id
final_parcel_df = final_parcel_df[fields].copy()
final_parcel_df['parcel_id_REMM_old'] = final_parcel_df['parcel_id_REMM']

# generate unique building id for split parcels
final_parcel_df.loc[(final_parcel_df['Split'].isna() == True), 'Split'] = 0
split_parcels = final_parcel_df[final_parcel_df['Split'] == 1].copy()
nonsplit_parcels = final_parcel_df[(final_parcel_df['Split']==0) | (final_parcel_df['Split']==2)].copy()
count = final_parcel_df.shape[0]
split_parcels['parcel_id_REMM'] = count + split_parcels.index + 1

# merge parcels back together once id has been figured out
final_parcel_df = pd.concat([nonsplit_parcels, split_parcels])
del nonsplit_parcels
del split_parcels

# set residential unit count of new parcels resulting from split to 0
final_parcel_df.loc[(final_parcel_df['Split']==1), 'UNIT_COUNT'] = 0

# set parcels with null unit counts to 0
final_parcel_df.loc[(final_parcel_df['UNIT_COUNT'].isna()), 'UNIT_COUNT'] = 0

# recalc land value and total market value using split factor
final_parcel_df.loc[(final_parcel_df['Split']==1), 'LAND_MKT_VALUE'] = final_parcel_df['LAND_MKT_VALUE'] / final_parcel_df['Split_Factor']
final_parcel_df.loc[(final_parcel_df['Split']==1), 'TOTAL_MKT_VALUE'] = final_parcel_df['LAND_MKT_VALUE']
final_parcel_df.loc[(final_parcel_df['Split']==2), 'TOTAL_MKT_VALUE'] = (final_parcel_df['TOTAL_MKT_VALUE'] - final_parcel_df['LAND_MKT_VALUE']) + (final_parcel_df['LAND_MKT_VALUE'] / final_parcel_df['Split_Factor'])
final_parcel_df.loc[(final_parcel_df['Split']==2), 'LAND_MKT_VALUE'] = final_parcel_df['LAND_MKT_VALUE'] / final_parcel_df['Split_Factor']

In [51]:
# fix county ids
final_parcel_df.loc[final_parcel_df['COUNTY_NAME'] == 'Davis', 'COUNTY_ID'] = '11'
final_parcel_df.loc[final_parcel_df['COUNTY_NAME'] == 'Salt Lake', 'COUNTY_ID'] = '35'
final_parcel_df.loc[final_parcel_df['COUNTY_NAME'] == 'Weber', 'COUNTY_ID'] = '57'

# add type column
final_parcel_df.loc[(final_parcel_df['building_type_id']==0), 'TYPE'] = 'Empty Buildable'
final_parcel_df.loc[(final_parcel_df['building_type_id']==1), 'TYPE'] = 'Single Family Res'
final_parcel_df.loc[(final_parcel_df['building_type_id']==2), 'TYPE'] = 'Multi Family Res'
final_parcel_df.loc[(final_parcel_df['building_type_id']==3), 'TYPE'] = 'Industrial'
final_parcel_df.loc[(final_parcel_df['building_type_id']==4), 'TYPE'] = 'Retail'
final_parcel_df.loc[(final_parcel_df['building_type_id']==5), 'TYPE'] = 'Office'
final_parcel_df.loc[(final_parcel_df['building_type_id']==6), 'TYPE'] = 'Governmental'
final_parcel_df.loc[(final_parcel_df['building_type_id']==7), 'TYPE'] = 'Mixed Use'
final_parcel_df.loc[(final_parcel_df['building_type_id']==8), 'TYPE'] = 'Other'
final_parcel_df.loc[(final_parcel_df['building_type_id']==9), 'TYPE'] = 'Educational'
final_parcel_df.loc[(final_parcel_df['building_type_id']==10), 'TYPE'] = 'Church'
final_parcel_df.loc[(final_parcel_df['building_type_id']==11), 'TYPE'] = 'Group Quarters'
final_parcel_df.loc[(final_parcel_df['building_type_id']==12), 'TYPE'] = 'Commons'
final_parcel_df.loc[(final_parcel_df['building_type_id']==13), 'TYPE'] = 'Healthcare'
final_parcel_df.loc[(final_parcel_df['building_type_id']==14), 'TYPE'] = 'Open Space Non-Buildable'
final_parcel_df.loc[(final_parcel_df['building_type_id']==15), 'TYPE'] = 'Agriculture'
final_parcel_df.loc[(final_parcel_df['building_type_id']==16), 'TYPE'] = 'Utilities'
final_parcel_df.loc[(final_parcel_df['building_type_id']==99), 'TYPE'] = 'No Build'

# make fields REMM compatible
final_parcel_df = final_parcel_df.rename(columns={"LAND_MKT_VALUE": "land_value"})
final_parcel_df = final_parcel_df.rename(columns={"COUNTY_NAME": "CO_NAME"})
final_parcel_df = final_parcel_df.rename(columns={"COUNTY_ID": "county_id"})
final_parcel_df = final_parcel_df.rename(columns={"BLDG_SQFT": "building_sqft"})
final_parcel_df = final_parcel_df.rename(columns={"UNIT_COUNT": "residential_units"})
final_parcel_df = final_parcel_df.rename(columns={"BUILT_YR": "year_built"})
final_parcel_df = final_parcel_df.rename(columns={"PARCEL_ACRES": "parcel_acres"})
final_parcel_df = final_parcel_df.rename(columns={"TYPE": "building_type"})

## Use spatial joins to get missing attribute values

In [52]:
# export only the shape and ids for spatial joining (~13 min)
parcels_for_sj_df= final_parcel_df[['parcel_id_REMM','max_dua', 'SHAPE']].copy()
parcels_for_sj = os.path.join(gdb,'_022_parcels_id_only')
parcels_for_sj_df.spatial.to_featureclass(location=parcels_for_sj, sanitize_columns=False)
del parcels_for_sj_df

In [53]:
# use spatial join to retrieve max dua
target_features = parcels_for_sj
join_features = gflu
output_features = os.path.join(gdb, "_023_parcels_gflu_2019_sj")

fieldmappings = arcpy.FieldMappings()
fieldmappings.addTable(target_features)
fieldmappings.addTable(join_features)

# attribute to summarize
fieldindex = fieldmappings.findFieldMapIndex('MaxDUA')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'Max'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# run the spatial join
sj = arcpy.SpatialJoin_analysis(target_features, join_features, output_features,'JOIN_ONE_TO_ONE', "KEEP_ALL", 
                           fieldmappings, "HAVE_THEIR_CENTER_IN")

In [54]:
gflu_old = r'E:\Projects\REMM-Input-Data-Prep-2019\Parcels\2020-WFRC\Inputs\Old_Policy.gdb\max_dua_rtp2015'

# use spatial join to retrieve old max dua
target_features = parcels_for_sj
join_features = gflu_old
output_features = os.path.join(gdb, "_023_parcels_gflu_2015_sj")

fieldmappings = arcpy.FieldMappings()
fieldmappings.addTable(target_features)
fieldmappings.addTable(join_features)

# attribute to summarize
fieldindex = fieldmappings.findFieldMapIndex('MaxDUA')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'Max'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# run the spatial join
sj4 = arcpy.SpatialJoin_analysis(target_features, join_features, output_features,'JOIN_ONE_TO_ONE', "KEEP_ALL", 
                           fieldmappings, "HAVE_THEIR_CENTER_IN")

In [55]:
# use spatial join  to get tazid from 8.3.2 traffic analysis zones
target_features = parcels_for_sj
join_features = old_taz
output_features = os.path.join(gdb, "_023_parcels_taz832_sj")

fieldmappings = arcpy.FieldMappings()
fieldmappings.addTable(target_features)
fieldmappings.addTable(join_features)

# attribute to summarize
fieldindex = fieldmappings.findFieldMapIndex('TAZID')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'First'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# run the spatial join
sj2 = arcpy.SpatialJoin_analysis(target_features, join_features, output_features,'JOIN_ONE_TO_ONE', "KEEP_ALL", 
                           fieldmappings, "HAVE_THEIR_CENTER_IN")

In [56]:
# use spatial join to get tazid from 9.0.0 traffic analysis zones
target_features = parcels_for_sj
join_features = new_taz
output_features = os.path.join(gdb, "_023_parcels_taz900_sj")

fieldmappings = arcpy.FieldMappings()
fieldmappings.addTable(target_features)
fieldmappings.addTable(join_features)

# attribute to summarize
fieldindex = fieldmappings.findFieldMapIndex('TAZID')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'First'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# run the spatial join
sj3 = arcpy.SpatialJoin_analysis(target_features, join_features, output_features,'JOIN_ONE_TO_ONE', "KEEP_ALL", 
                           fieldmappings, "HAVE_THEIR_CENTER_IN")

In [57]:
# use spatial join to get census tract id
target_features = parcels_for_sj
join_features = census_tracts
output_features = os.path.join(gdb, "_023_parcels_tract_sj")

fieldmappings = arcpy.FieldMappings()
fieldmappings.addTable(target_features)
fieldmappings.addTable(join_features)

# attribute to summarize
fieldindex = fieldmappings.findFieldMapIndex('GEOID')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'First'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# run the spatial join
sj6 = arcpy.SpatialJoin_analysis(target_features, join_features, output_features,'JOIN_ONE_TO_ONE', "KEEP_ALL", 
                           fieldmappings, "HAVE_THEIR_CENTER_IN")

In [58]:
# use spatial join to get missing square footage from old data
target_features = parcels_for_sj
join_features = nonres_pts
output_features = os.path.join(gdb, "_023_parcels_sqft_sj")

fieldmappings = arcpy.FieldMappings()
fieldmappings.addTable(target_features)
fieldmappings.addTable(join_features)

# attribute to summarize
fieldindex = fieldmappings.findFieldMapIndex('old_nonres_sqft')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'Sum'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# run the spatial join
sj5 = arcpy.SpatialJoin_analysis(target_features, join_features, output_features,'JOIN_ONE_TO_ONE', "KEEP_ALL", 
                           fieldmappings, "INTERSECT")
                           

In [59]:
# max far
target_features = parcels_for_sj
join_features = centers
output_features = os.path.join(gdb, "_023_parcels_centers_sj")

fieldmappings = arcpy.FieldMappings()
fieldmappings.addTable(target_features)
fieldmappings.addTable(join_features)

# attribute to summarize
fieldindex = fieldmappings.findFieldMapIndex('MaxFAR')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'Max'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# run the spatial join
sj7 = arcpy.SpatialJoin_analysis(target_features, join_features, output_features,'JOIN_ONE_TO_ONE', "KEEP_ALL", 
                           fieldmappings, "INTERSECT")

In [60]:
# use spatial join to get local FAR for calcuating missing square footage
target_features = parcels_for_sj
join_features = base_year_surface1
output_features = os.path.join(gdb, "_023_parcels_bys1_sj")

fieldmappings = arcpy.FieldMappings()
fieldmappings.addTable(target_features)
fieldmappings.addTable(join_features)

fieldindex = fieldmappings.findFieldMapIndex('FAR_SF')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'Max'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

fieldindex = fieldmappings.findFieldMapIndex('FAR_MF')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'Max'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

fieldindex = fieldmappings.findFieldMapIndex('FAR_IND')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'Max'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

fieldindex = fieldmappings.findFieldMapIndex('FAR_NONIND')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'Max'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# run the spatial join
sj9 = arcpy.SpatialJoin_analysis(target_features, join_features, output_features,'JOIN_ONE_TO_ONE', "KEEP_ALL", 
                           fieldmappings, "HAVE_THEIR_CENTER_IN")

In [61]:
# use spatial join to get local FAR for calcuating missing square footage
target_features = parcels_for_sj
join_features = base_year_surface2
output_features = os.path.join(gdb, "_023_parcels_bys2_sj")

fieldmappings = arcpy.FieldMappings()
fieldmappings.addTable(target_features)
fieldmappings.addTable(join_features)

# attribute to summarize
fieldindex = fieldmappings.findFieldMapIndex('MAXDUA_MED')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'Max'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

fieldindex = fieldmappings.findFieldMapIndex('YB_SF')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'First'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

fieldindex = fieldmappings.findFieldMapIndex('YB_MF')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'First'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

fieldindex = fieldmappings.findFieldMapIndex('YB_IND')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'First'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

fieldindex = fieldmappings.findFieldMapIndex('YB_NONIND')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'First'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# run the spatial join
sj10 = arcpy.SpatialJoin_analysis(target_features, join_features, output_features,'JOIN_ONE_TO_ONE', "KEEP_ALL", 
                           fieldmappings, "HAVE_THEIR_CENTER_IN")

In [62]:
# read in and merge spatial join tables together (~8 min)
sj_df = pd.DataFrame.spatial.from_featureclass(sj[0])[['parcel_id_REMM', 'MaxDUA']].copy()
sj_df.columns = ['parcel_id_REMM', 'MaxDUA_New']

sj4_df = pd.DataFrame.spatial.from_featureclass(sj4[0])[['parcel_id_REMM', 'MaxDua']].copy()
sj4_df.columns = ['parcel_id_REMM', 'MaxDUA_Old']

sj2_df = pd.DataFrame.spatial.from_featureclass(sj2[0])[['parcel_id_REMM', 'TAZID']].copy()
sj2_df.columns = ['parcel_id_REMM', 'TAZID_832']

sj3_df = pd.DataFrame.spatial.from_featureclass(sj3[0])[['parcel_id_REMM', 'TAZID']].copy()
sj3_df.columns = ['parcel_id_REMM', 'TAZID_900']

sj6_df = pd.DataFrame.spatial.from_featureclass(sj6[0])[['parcel_id_REMM', 'GEOID']].copy()
sj6_df.columns = ['parcel_id_REMM', 'Tract_GEOID']

sj5_df = pd.DataFrame.spatial.from_featureclass(sj5[0])[['parcel_id_REMM', 'old_nonres_sqft']].copy()
sj5_df.columns = ['parcel_id_REMM', 'old_nonres_sqft']

sj7_df = pd.DataFrame.spatial.from_featureclass(sj7[0])[['parcel_id_REMM', 'MaxFAR']].copy()
sj7_df.columns = ['parcel_id_REMM', 'MaxFAR_New']

sj9_df = pd.DataFrame.spatial.from_featureclass(sj9[0])[['parcel_id_REMM', 'FAR_SF','FAR_MF','FAR_IND','FAR_NONIND']].copy()

sj10_df = pd.DataFrame.spatial.from_featureclass(sj10[0])[['parcel_id_REMM', 'MAXDUA_MED', 'YB_SF', 'YB_MF', 'YB_IND', 'YB_NONIND']].copy()

### Set Max DUA

In [63]:
# track null duas
print(final_parcel_df[final_parcel_df['max_dua'].isna()].shape)

# max dua from 2019
merged = final_parcel_df.merge(sj_df, left_on='parcel_id_REMM', right_on='parcel_id_REMM', how='left')
merged = merged.merge(sj10_df, left_on='parcel_id_REMM', right_on='parcel_id_REMM', how='left')

del final_parcel_df
del sj10_df
del sj_df

merged['max_dua'] = merged['MaxDUA_New']
del merged['MaxDUA_New']
print(merged[merged['max_dua'].isna()].shape)

# max dua from 2015
merged = merged.merge(sj4_df, left_on='parcel_id_REMM', right_on='parcel_id_REMM', how='left')
del sj4_df
merged.loc[(merged['max_dua'].isna()==True) & (merged['MaxDUA_Old'] >= 0), 'max_dua'] = merged['MaxDUA_Old']
del merged['MaxDUA_Old']
print(merged[merged['max_dua'].isna()].shape)

# set max dua of open space parcels to 0
merged.loc[(merged['building_type_id']==14), 'max_dua'] = 0

# set max dua using zoning surface
merged.loc[(merged['max_dua'].isna()==True) & (merged['MAXDUA_MED'] >= 0), 'max_dua'] = merged['MAXDUA_MED']
del merged['MAXDUA_MED']

# set zeros to null
merged.loc[(merged['max_dua'] == 0), 'max_dua'] = np.nan

(671812, 37)
(2228, 42)
(1689, 42)


### Add missing "year_built"

In [64]:
merged.loc[(((merged['year_built'].isna()==True) | (merged['year_built'] <= 0)) & (merged['building_type_id'].isin([1]))), 'year_built'] = merged['YB_SF']
merged.loc[(((merged['year_built'].isna()==True) | (merged['year_built'] <= 0)) & (merged['building_type_id'].isin([2]))), 'year_built'] = merged['YB_MF']
merged.loc[(((merged['year_built'].isna()==True) | (merged['year_built'] <= 0)) & (merged['building_type_id'].isin([3]))), 'year_built'] = merged['YB_IND']
merged.loc[(((merged['year_built'].isna()==True) | (merged['year_built'] <= 0)) & (merged['building_type_id'].isin([4,5,6,7,8,9,10,11,13]))), 'year_built'] = merged['YB_NONIND']


del merged['YB_SF']
del merged['YB_MF']
del merged['YB_IND']
del merged['YB_NONIND']

### Set Max FAR

In [65]:
merged = merged.merge(sj7_df, left_on='parcel_id_REMM', right_on='parcel_id_REMM', how='left')
del sj7_df
merged.loc[(merged['MaxFAR_New'] > 0), 'max_far'] = merged['MaxFAR_New']
del merged['MaxFAR_New']

# set non residential parcels
merged.loc[((merged['max_far'].isna()==True) & (merged['type1'] != 1) & (merged['type2'] != 1)), 'max_far'] = .5
        
# set industrial only parcels
merged.loc[((merged['type3'] == 1) & (merged['type1'] == 1) & (merged['type2'] == 1) & (merged['type4'] == 1) &
            (merged['type5'] == 1) & (merged['type6'] == 1) & (merged['type7'] == 1) & (merged['type8'] == 1) & (merged['max_far'].isna()==True)), 'max_far'] = .41

# set max far of open space parcels to 0
merged.loc[(merged['building_type_id']==14), 'max_far'] = 0

# set zeros to null
merged.loc[(merged['max_far'] == 0), 'max_far'] = np.nan

### Add missing "building_sqft" from 2015

In [66]:
merged = merged.merge(sj5_df, left_on='parcel_id_REMM', right_on='parcel_id_REMM', how='left')
del sj5_df

print(merged[(merged['building_sqft'].isna()) |  (merged['building_sqft'] <= 0)].shape)
merged.loc[(((merged['building_sqft'].isna()==True) | (merged['building_sqft'] <= 0)) & ~(merged['building_type_id'].isin([1,2]))), 'building_sqft'] = merged['old_nonres_sqft']
print(merged[(merged['building_sqft'].isna()) |  (merged['building_sqft'] <= 0)].shape)
del merged['old_nonres_sqft']

(150781, 38)
(148158, 38)


In [67]:
merged = merged.merge(sj9_df, left_on='parcel_id_REMM', right_on='parcel_id_REMM', how='left')
del sj9_df

merged.loc[(((merged['building_sqft'].isna()==True) | (merged['building_sqft'] <= 0)) & (merged['building_type_id'].isin([1]))), 'building_sqft'] = merged['FAR_SF'] * merged['parcel_acres']
merged.loc[(((merged['building_sqft'].isna()==True) | (merged['building_sqft'] <= 0)) & (merged['building_type_id'].isin([2]))), 'building_sqft'] = merged['FAR_MF'] * merged['parcel_acres']
merged.loc[(((merged['building_sqft'].isna()==True) | (merged['building_sqft'] <= 0)) & (merged['building_type_id'].isin([3]))), 'building_sqft'] = merged['FAR_IND'] * merged['parcel_acres']
merged.loc[(((merged['building_sqft'].isna()==True) | (merged['building_sqft'] <= 0)) & (merged['building_type_id'].isin([4,5,6,7,8,9,10,11,13]))), 'building_sqft'] = merged['FAR_NONIND'] * merged['parcel_acres']

merged['building_sqft'] = merged['building_sqft'].fillna(0)
merged['building_sqft'] = merged['building_sqft'].astype(int)


del merged['FAR_SF']
del merged['FAR_MF']
del merged['FAR_IND']
del merged['FAR_NONIND']

### Add zone ids

In [68]:
# taz832
merged = merged.merge(sj2_df, left_on='parcel_id_REMM', right_on='parcel_id_REMM', how='left')
del sj2_df

# taz 900
merged = merged.merge(sj3_df, left_on='parcel_id_REMM', right_on='parcel_id_REMM', how='left')
del sj3_df

# Census Tract GEOID
merged = merged.merge(sj6_df, left_on='parcel_id_REMM', right_on='parcel_id_REMM', how='left')
del sj6_df


### Final export

In [69]:
merged.columns

Index(['parcel_id_REMM', 'CO_NAME', 'TAZID', 'county_id', 'PARCEL_ID',
       'TOTAL_MKT_VALUE', 'land_value', 'residential_units', 'building_sqft',
       'FLOORS_CNT', 'year_built', 'EFFBUILT_YR', 'IS_OUG', 'max_dua',
       'max_far', 'max_height', 'type1', 'type2', 'type3', 'type4', 'type5',
       'type6', 'type7', 'type8', 'agriculture', 'basebldg', 'NoBuild',
       'redev_friction', 'building_type_id', 'x', 'y', 'parcel_acres', 'Split',
       'Split_Factor', 'SHAPE', 'parcel_id_REMM_old', 'building_type',
       'TAZID_832', 'TAZID_900', 'Tract_GEOID'],
      dtype='object')

In [70]:
merged.spatial.to_featureclass(location=os.path.join(gdb2,'parcels_2019'), sanitize_columns=False)

'e:\\Projects\\REMM-Input-Data-Prep-2019\\Parcels\\2020-WFRC\\Outputs\\remm_base_year_2019.gdb\\parcels_2019'

In [71]:
# convert residential parcels to pts for popsim
final_parcels_lyr = arcpy.MakeFeatureLayer_management(os.path.join(gdb2,'parcels_2019'), 'final_parcels_lyr', where_clause=""" building_type_id IN (1, 2) """) 
arcpy.FeatureToPoint_management(final_parcels_lyr, os.path.join(gdb2,'residential_centroids_2019'), "INSIDE")

<Result '.\\Outputs\\remm_base_year_2019.gdb\\residential_centroids_2019'>

In [72]:
# check for duplicate ids
id_summary = pd.DataFrame(merged['parcel_id_REMM'].value_counts()).reset_index()
id_summary.columns = ['id','count']
dups = id_summary[id_summary['count']>1]
dups_ids = dups['id'].to_list()
pdups = merged[merged['parcel_id_REMM'].isin(dups_ids)]
pdups = pdups[['parcel_id_REMM','Split_Factor', 'SHAPE']].copy()
print(pdups.shape)
pdups.spatial.to_featureclass(location=os.path.join(gdb,'_024_duplicates'))
del id_summary

(0, 3)


In [73]:
os.startfile(r".\Review.aprx")